# 0.0 IMPORTS

In [83]:
import pandas as pd
import inflection
import math

## Helper Functions

## Load Data

In [84]:
df_sales_raw = pd.read_csv('Data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('Data/store.csv', low_memory=False)

#unindo as duas tabelas através de um left join utilizando a chave estrangeira Store

df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')


low_memory = False -> Os dados são carregados de uma vez só, caso seja False, os dados são carregados em partes.

In [85]:

df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
724359,395,5,2013-09-20,3414,479,1,0,0,0,a,a,3620.0,2.0,2013.0,0,NaN,NaN,NaN


# 1.0 DESCRIÇÃO DOS DADOS

## 1.1 Rename Columns

In [86]:
df1 = df_raw.copy()

In [87]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

In [88]:
#trocando as colunas para snake_case
snake_case = lambda x: inflection.underscore(x)

cols_new = list(map(snake_case, cols_old))

# rename columns

df1.columns = cols_new

In [89]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Data Dimension

In [90]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of rows: 1017209
Number of columns: 18


## 1.3 Data Types

In [91]:
df1['date'] = pd.to_datetime(df1['date'])

In [92]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NA's

In [93]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

Existem três maneira de se tratara um NA 
1. Excluir os NA's: Vantagem é uma forma rápida, desvantagem é que estamos jogando os dados fora. Principalmente se o resto das colunas tem um valor importante para o negócio
2. Utilizando algoritímos de ML: substituindo as colunas pelo comportamento da coluna, ou seja, calculo da mediana, média, algoritímo para prever qual o valor que estaria no local.
3. Através do entedimento do negócio: Será que é uma regra ou lógica de negócio definido pelo time de negócios?

## 1.5 Fillout NA

#### 1.5.1 competition_distance: distance in meters to the nearest competitor store

- A partir do entendimento do negócio, assumimos que os valores faltantes nesta coluna são consequências de não haver lojas que realizam competição em distâncias curtas, ou a distância é muito grande que nem é computada no dataset. Por isso, a opção por substituir os NA's desta coluna por uma distância muito maior que a distância máxima do meu dataset parece razoável.

In [94]:
df1['competition_distance'].max()

75860.0

In [95]:
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.00 if math.isnan(x) else x)

#### 1.5.2 competition_open_since_month: gives the approximate year and month of the time the nearest competitor was opened
- competition_open_since_year

- Aqui o valor pode estar nulo pela loja concorrente ter aberto antes da minha loja e eu não ter essa data, ou abriu depois e esquecemos de anotar. Para preencher esse NA levamos em conta etapas do feature engineering em que o tempo é muito importante. A solução é pegar a data da última venda.

In [96]:
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

In [97]:
                       
# promo2_since_week  
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year  
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)
             
          

#### 1.5.3 Promo_interval
- A coluna indica uma promoção continua nas lojas, se o valor é nulo quer dizer que a loja não esta realizando promoção

    

In [98]:
month_map = {1 : 'Jan', 
 2 : 'Feb', 
 3 : 'Mar',
 4 : 'Apr',
 5 : 'May',
 6 : 'Jun',
 7 : 'Jul',
 8 : 'Aug',
 9 : 'Sept',
 10 : 'Oct',
 11 : 'Nov',
 12 : 'Dec'}

In [99]:
df1['promo_interval'].fillna(0, inplace=True)    

In [100]:
df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x : 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'] else 0, axis=1)

In [101]:
df1.sample(5).T

,979450,144704,705424,841736,439195
store,151,870,415,697,671
day_of_week,7,2,1,5,2
date,2013-02-03 00:00:00,2015-03-24 00:00:00,2013-10-07 00:00:00,2013-06-07 00:00:00,2014-06-03 00:00:00
sales,0,6731,7308,11698,7639
customers,0,864,609,1073,704
open,0,1,1,1,1
promo,0,0,1,1,1
state_holiday,0,0,0,0,0
school_holiday,0,0,1,0,0
store_type,d,a,d,d,a


In [102]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

## 1.6 

## 1.7

## 1.8